# Chapter 3 The Data Encryption Standard (DES) and Alternatives

## 3.1 Introduction to DES

There are two primitive operations with which strong encryption algorithms can be built:
1. **Confusion** is an encryption operation where the relationship between key and ciphertext is obscured. Today, a common element for achieving confusion is substitution, which is found in both DES and AES.  
2. **Diffusion** is an encryption operation where the influence of one plaintext symbol is spread over many ciphertext symbols with the goal of hiding statistical properties of the plaintext. A simple diffusion element is the bit permutation, which is used frequently within DES. AES uses the more advanced MixColumn operation.


Modern block ciphers possess excellent diffusion properties. On a cipher level this means that changing of one bit of plaintext results on average in the change of half the output bits

## 3.2 Overview of the DES Algorithm

DES is a cipher which encrypts *blocks of length of 64 bits* with a *key of size of 56 bits*.

* Symmetric cipher, i.e., the same same key is used for encryption and decryption
* Iterative algorithm. For each block of plaintext, encryption is handled in 16 rounds which all perform the identical operation.
* In every round a different subkey is used and all subkeys $k_i$ are derived from the main key $k$.

### Feistel network

* Used in many, but certainly not in all, modern block ciphers. (AES is not a Feistel cipher.)
* In each round, a round key $k_i$ is derived from the main 56-bit key using what is called the key schedule.
* Encryption and decryption are almost the same operation. Decryption requires only a reversed key schedule.

## 3.3 Internal Structure of DES

### Initial and Final Permutation

Initial permutation $IP$  
Final permutation $IP^{-1}$

In [34]:
def reverse_permutation(p):
    result = [None] * len(p)
    for i,j in enumerate(p):
        result[j-1] = i+1
    return result

INITIAL_PERMUTATION_TABLE = [
    58 , 50 , 42 , 34 , 26 , 18 , 10 , 2,
    60 , 52 , 44 , 36 , 28 , 20 , 12 , 4,
    62 , 54 , 46 , 38 , 30 , 22 , 14 , 6, 
    64 , 56 , 48 , 40 , 32 , 24 , 16 , 8, 
    57 , 49 , 41 , 33 , 25 , 17 , 9 , 1,
    59 , 51 , 43 , 35 , 27 , 19 , 11 , 3,
    61 , 53 , 45 , 37 , 29 , 21 , 13 , 5,
    63 , 55 , 47 , 39 , 31 , 23 , 15 , 7]

FINAL_PERMUTATION_TABLE = reverse_permutation(INITIAL_PERMUTATION_TABLE)

### The $f$-Function

* **Expansion permutation** the 32-bit input is expanded to 48 bits by partitioning the input into eight 4-bit blocks and by expanding each block to 6 bits.
* **Substition boxes (S-boxes)** the most significant bit (MSB) and the least significant bit (LSB) of each 6-bit input select the row of the table, while the four inner bits select the column. 
* **$P$ permutation** the 32-bit output is permuted bitwise according to the $P$ permutation.


**S-boxes**
* The core of DES in terms of cryptographic strength. 
* The only **nonlinear element** in the algorithm and provide confusion.
* Carefully designed to also thwart advanced mathematical attacks, in particular **differential cryptanalysis**.


The diffusion caused by the expansion, S-boxes and the permutation $P$ guarantees that every bit at the end of the fifth round is a function of every plaintext bit and every key bit. This behavior is known as the **avalanche effect**.

In [32]:
EXPANSION_PERMUTATION_TABLE = [
    32,1,2,3,4,5,
    4,5,6,7,8,9,
    8,9,10,11,12,13,
    12,13,14,15,16,17,
    16,17,18,19,20,21,
    20,21,22,23,24,25,
    24,25,26,27,28,29,
    28,29,30,31,32,1]

SBOX = [
    # Box-1
    [
    [14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
    [0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
    [4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
    [15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]
    ],
    # Box-2
    [
    [15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
    [3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
    [0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
    [13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]
    ],
    # Box-3
    [
    [10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
    [13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
    [13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
    [1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]
    ],
    # Box-4
    [
    [7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
    [13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
    [10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
    [3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]
    ],
    # Box-5
    [
    [2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
    [14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
    [4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
    [11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]
    ],
    # Box-6
    [
    [12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
    [10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
    [9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
    [4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]
    ],
    # Box-7
    [
    [4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
    [13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
    [1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
    [6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]
    ],
    # Box-8
    [
    [13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
    [1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
    [7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
    [2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]
    ]
]

### Key Schedule

DES input key is often stated as 64-bit, where every eighth bit is used as an odd parity bit over the preceding seven bits. Actually **56-bit** cipher.

**PC−1 permutation**
* The 64-bit key is first reduced to 56 bits by ignoring every eighth bit.
* PC−1 stands for "permuted choice one".

The resulting 56-bit key is split into two halves C0 and D0
The two 28-bit halves are cyclically shifted left by one or two bit positions:
* $i = 1,2,9,16$, rotated left by one bit.
* $i \neq 1,2,9,16$, rotated left by two bits.

Then, $C_0 = C_{16}$ and $D_0 = D_{16}$

To derive the 48-bit round keys $k_i$, the two halves are permuted bitwise again with PC−2 (permuted choice 2).

In [33]:
PC1 = [
    57,49,41,33,25,17,9,1,
    58,50,42,34,26,18,10,2,
    59,51,43,35,27,19,11,3,
    60,52,44,36,63,55,47,39,
    31,23,15,7,62,54,46,38,
    30,22,14,6,61,53,45,37,
    29,21,13,5,28,20,12,4]

PC2 = [
    14,17,11,24,1,5,3,28,
    15,6,21,10,23,19,12,4,
    26,8,16,7,27,20,13,2,
    41,52,31,37,47,55,30,40,
    51,45,33,48,44,49,39,56,
    34,53,46,42,50,36,29,32
]

## 3.4 Decryption

Only the key schedule is reversed, i.e., in decryption round 1, subkey 16 is needed; in round 2, subkey 15; etc.

Given the initial DES key $k$, can we easily generate $k_{16}$?

\begin{equation}
\begin{split}
k_{16} &= PC2(C_{16},D_{16}) \\
    &= PC2(C_0,D_0) \\
    &= PC2(PC1(k))
\end{split}
\end{equation}

\begin{equation}
\begin{split}
k_{15} &= PC2(C_{15},D_{15}) \\
&= PC2(RS2(C_{16}),RS2(D_{16})) \\
&= PC2(RS2(C_0),RS2(D_0))
\end{split}
\end{equation}

* In decryption round 1, not rotated.
* In decryption rounds 2, 9 and 16, rotated right by one bit.
* In the other rounds, rotated right by two bits.

### Decryption in Feistel Networks

The basic idea is that the decryption function reverses the DES encryption in a round-by-round manner. That means that decryption round 1 reverses encryption round 16, decryption round 2 reverses encryption round 15, and so on.

## 3.5 Security of DES

1. The key space is too small, i.e., the algorithm is vulnerable against brute-force attacks.
2. The design criteria of the S-boxes was kept secret and there might have existed an analytical attack that exploits mathematical properties of the S-boxes, but which is only known to the DES designers.  
DES is quite robust against known analytical attacks: In practice it is very difficult to break the cipher with **differential cryptanalysis** or **linear cryptanalysis**.

In 2006, the COPACOBANA (Cost-Optimized Parallel Code-Breaker) machine allows one to break DES with an average search time of less than 7 days with costs in the \$10,000 range.

In summary, a key size of 56 bits is **too short** to encrypt confidential data nowadays. 

only short-term security is needed (a few hours, or where the value of the encrypted data is very low.

## 3.7 DES Alternatives

* **Advanced Encryption Standard (AES)**
* **Triple DES (3DES)**  
  $y = DES_{k_3}(DES_{k_2} (DES_{k_1}(x)))$  
  $y = DES_{k_3}(DES^{-1}_{k_2} (DES_{k_1}(x)))$ (performs single DES encryption if $k_3 = k_2 = k_1$)
* **Key whitening**. Two additional 64-bit keys $k_1$ and $k_2$ are XORed to the plaintext and ciphertext, respectively, prior to and after the DES algorithm.  
  $y = DES_{k,k_1,k_2}(x) = DES_k(x \bigoplus k_1) \bigoplus k_2$

## 3.9 Lessons Learned

* DES was the dominant symmetric encryption algorithm from the mid-1970s to the mid-1990s. Since 56-bit keys are no longer secure, the Advanced Encryption Standard (AES) was created.
* Standard DES with 56-bit key length can be broken relatively easily nowadays through an exhaustive key search.
* DES is quite robust against known analytical attacks: In practice it is very difficult to break the cipher with differential or linear cryptanalysis.
* DES is reasonably efficient in software and very fast and small in hardware. By encrypting with DES three times in a row, triple DES (3DES) is created, against which no practical attack is currently known.
* The "default" symmetric cipher is nowadays often AES. In addition, the other four AES finalist ciphers all seem very secure and efficient.
* Since about 2005 several proposals for lightweight ciphers have been made. They are suited for resource-constrained applications.